In [ ]:
import os
import pandas as pd
from pandasai import Agent

# Sample DataFrame
epl_dataframe = pd.read_csv("data.csv")

display(epl_dataframe)
# By default, unless you choose a different LLM, it will use BambooLLM.
# You can get your free API key signing up at https://pandabi.ai (you can also configure it in your .env file)
os.environ["PANDASAI_API_KEY"] = "$2a$10$Acl/GiigqUBVrkIto70yQOgGCTGxDMZcdVkws/q12g4O54p3y55oW"

agent = Agent(epl_dataframe)
agent.chat('Which home or away teams are most likely to win?')


In [ ]:
agent.chat("Can you construct the graph of HomeWinOdds vs DrawOdds")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Cox-Ingersoll-Ross (CIR) Model parameters
K = 0.9           # Speed of mean reversion
long_mean = 4     # Long-term mean
variance = 3      # Volatility

N = 100  # Number of simulations
T = 1    # Total time (1 year)
M = 400  # Number of time steps
d_t = T / M  # Time step size

# Initialize the simulation matrix
corner_sim = np.zeros((M, N))
initial_value = 4  # Initial value for each simulation
corner_sim[0, :] = initial_value

# Generate all random changes at once
random_changes = np.sqrt(d_t) * np.random.normal(0, 1, (M-1, N))

# Vectorized simulation of the CIR process
for t in range(1, M):
    corner_sim[t, :] = (corner_sim[t-1, :] +
                        K * (long_mean - corner_sim[t-1, :]) * d_t +
                        variance * np.sqrt(np.maximum(corner_sim[t-1, :], 0)) * random_changes[t-1, :])

# Plot the first 10 simulations
plt.plot(corner_sim[:, :10])
plt.ylabel("Value of Corners")
plt.xlabel("Days")
plt.title("Monte Carlo Simulation of Corners Using CIR Model (Vectorized)")
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def heston_model_simulation(S0, v0, rho, kappa, theta, sigma, T, N, M):
    dt = T / N
    mu = np.array([0, 0])
    cov = np.array([[1, rho],
                    [rho, 1]])

    S = np.full(shape=(N + 1, M), fill_value=S0)
    v = np.full(shape=(N + 1, M), fill_value=v0)
    m = np.full(shape=(N + 1, M), fill_value=0.2)

    Z = np.random.multivariate_normal(mu, cov, (N, M))
    for i in range(1, N + 1):
        S[i] = S[i - 1] * np.exp((r - q - 0.5 * v[i - 1]) * dt + np.sqrt(v[i - 1] * dt) * Z[i - 1, :, 0])
        v[i] = np.maximum(v[i - 1] + kappa * (theta - v[i - 1]) * dt + sigma * np.sqrt(v[i - 1] * dt) * Z[i - 1, :, 1],
                          0)
        m[i] = m[i - 1]*np.exp(-theta * Z[i - 1, :, 0])

    return S, v, m


if __name__ == "__main__":
    T = 1
    r = 0.01
    q = 0.02
    S0 = 2300
    kappa = 0.7
    rho_a = -0.60
    M = 5
    N = 252
    v0 = 0.20 ** 2
    theta = 0.15 ** 2
    sigma = 0.5

    S_n, v_n, M_n = heston_model_simulation(S0, v0, rho_a, kappa, theta, sigma, T, N, M)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 5))
    time = np.linspace(0, T, N + 1)
    ax1.plot(time, S_n)
    ax1.set_title('Asset Prices')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Asset Prices')
    ax2.plot(time, v_n)
    ax2.set_title('Volatility')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Volatility')
    ax3.plot(time, M_n)
    ax3.set_title('Risk Price')
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Risk Price')
    plt.show()

In [ ]:
import yfinance as yf

start_date = "2017-01-03"
end_date = "2019-10-01"
# Define tickers for US Treasury yields
treasury_tickers = ['^IRX', '^FVX', '^TNX', '^TYX']

# Fetch historical data (e.g., for the last year)
treasury_data = yf.download(treasury_tickers, start=start_date, end=end_date)

# Display historical closing prices for each tenor
print(treasury_data['Close'])

In [ ]:
import matplotlib.pyplot as plt

# Plot the closing yields
treasury_data['Close'].plot(figsize=(10, 6), title="U.S. Treasury Yields Across Tenors")
plt.ylabel("Yield (%)")
plt.show()

In [ ]:
Y = treasury_data['Close'].loc["2019-09-03"].to_list()  # Yields for each tenor on that date
X = ['^IRX', '^FVX', '^TNX', '^TYX']  # Tenors: 3-month, 5-year, 10-year, 30-year

# Plot yields for the specific date across tenors
plt.figure(figsize=(8, 5))
plt.plot(X, Y, marker='o', color='b', linestyle='-', linewidth=2, markersize=8)  # Line plot with markers
plt.title(f"U.S. Treasury Yields on {start_date}")
plt.xlabel("Tenors")
plt.ylabel("Yield (%)")
plt.grid(True)  # Optional: add gridlines for better readability
plt.show()

In [ ]:
from datetime import datetime
import numpy as np

time_days = datetime.strptime("2027-07-01", "%Y-%m-%d") - datetime.now()
time_days = time_days.days / 365
last_yield= 2.809/100
fv = 100
coupon_rate = 0.375/100

In [ ]:
def calculate_coupon(coupon_rate, face_value, time_in_years, yield_value, exponentially=False):
    coupon_values = []
    
    # Using numpy.arange to create a range of float values
    time_discount = 0.0
    while time_discount <= time_in_years:
        coupon_values.append(discounter(coupon_rate * face_value, yield_value, time_discount, exponentially))
        print(f"Time Discount: {time_discount}, Coupon Value: {coupon_values[-1]}")
        time_discount += 0.25  # Increment by 0.25 years

    return coupon_values 

def discounter(value, rate, time, exponentially=False):
    if exponentially:
        return value*np.exp(-rate*time)
    return value/np.power(rate+1, time)

In [ ]:
pv_of_coupons = calculate_coupon(coupon_rate, fv, time_days, last_yield, True)
pv_of_principal = discounter(fv, last_yield, time_days, True)

print(f"PV: {pv_of_principal}")

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

# Parameters
start_date = "2015-07-01"
end_date = "2023-12-31"
n_tenors = 20
tenors = np.linspace(3, 60, n_tenors, dtype=int)  # Tenors from 3 to 60 months

# Generate date range
date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # Business days

# Initialize DataFrame
data = []

# Populate dataset
for date in date_range:
    for tenor in tenors:
        if tenor == 3 and date < pd.to_datetime("2022-03-01"):
            # Use Eurodollar Futures data before March 2022 for 3-month tenor
            contract_type = "Eurodollar"
            price_variation = np.random.normal(0, 0.02)  # Example price variation
            order_flow = np.random.normal(0, 100)  # Example order flow
        else:
            # Use SOFR Futures data after March 2022 or for other tenors
            contract_type = "SOFR"
            price_variation = np.random.normal(0, 0.015)  # Example price variation
            order_flow = np.random.normal(0, 80)  # Example order flow

        # Append to data
        data.append([date, tenor, contract_type, price_variation, order_flow])

# Create DataFrame
df = pd.DataFrame(data, columns=["Date", "Tenor (Months)", "Contract Type", "Price Variation", "Net Order Flow"])

# Display sample
print(df.head())


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

FORMAT = "%d/%m/%Y"

def is_business_day(date, holidays=[]):
    date = pd.Timestamp(date)
    return (date.weekday() < 5) and (date not in holidays)

def next_business_day(date):
    date = pd.Timestamp(date)
    next_day = date + pd.offsets.BDay(1)
    return next_day

def __create_days_vector(start_date: datetime.date, settlement_date: datetime.date):
    current_date = start_date
    days_vector = []
    while current_date < settlement_date:
        if is_business_day(current_date):
            next_day = next_business_day(current_date)
            days_delta = (next_day - current_date).days
            days_vector.append(days_delta)
        current_date += timedelta(days=1)
    return days_vector

def calculate_r(start_date: str, settlement_date: str, r_vector, round_to=4) -> float:
    r_today = r_vector[-1]
    start_date = datetime.strptime(start_date, FORMAT)
    settlement_date = datetime.strptime(settlement_date, FORMAT)
    days_vector = __create_days_vector(start_date, settlement_date)
    
    if len(r_vector) == 1:
        r_vector = [r_today] * len(days_vector)
    
    days_in_reference_quarter = np.sum(days_vector)
    days_vector = np.array(days_vector)/360
    r_vector = np.array(r_vector)/100
    
    compounded_value = days_vector * r_vector + 1
    compounded_value = np.prod(compounded_value) - 1
    # print(compounded_value)
    return round(compounded_value * 360 * 100 / days_in_reference_quarter, round_to)

def get_pv01(rate:np.array, start_date, settlement_date, in_bps=True):
    R = calculate_r(start_date, settlement_date , rate)
    change = R - calculate_r(start_date, settlement_date, rate-0.01)
    print(R)
    print(calculate_r(start_date, settlement_date, rate-0.01))
    if in_bps:
        return change*10000/(100-R)
    return change*100/(100-R)


In [ ]:
settlement_date = "19/03/2025"
start_date = datetime.strftime(datetime.strptime(settlement_date, FORMAT) - pd.DateOffset(months=3), FORMAT)

R = calculate_r(start_date, settlement_date , [4.82])
imm_s3h5 = 100 - R
pv_01 = get_pv01(np.array([4.82]), start_date, settlement_date)
print(f"SOFR 3m for settlement date: {settlement_date} (START of contract: {start_date}) : {imm_s3h5} (R={R}) PV01: {round(pv_01, 2)} bps")

In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to fetch page content
def fetch_page(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page: {e}")
        return None

# Function to scrape match details
def scrape_match_details(url):
    page_content = fetch_page(url)
    if not page_content:
        return None

    soup = BeautifulSoup(page_content, 'html.parser')

    # Example: Extract match details
    try:
        match_details = {}
        match_details['teams'] = [team.text for team in soup.find_all('div', class_='team-name')]
        match_details['score'] = soup.find('div', class_='score').text.strip()
        match_details['date'] = soup.find('div', class_='date').text.strip()

        return match_details
    except AttributeError:
        print("Failed to parse match details. The website structure might have changed.")
        return None

# Example Usage
if __name__ == "__main__":
    # Replace this with the URL of the specific match or page on SofaScore
    sofascore_url = "https://www.sofascore.com/football/2024-11-28"
    match_data = scrape_match_details(sofascore_url)

    if match_data:
        print("Match Details:")
        for key, value in match_data.items():
            print(f"{key.capitalize()}: {value}")
    else:
        print("No match data found.")


In [2]:
test_string = "adsdewfdvrttttttdfdfvgbddddssssssaswsqwwwqefffefefggg"

def remove_k_consecutive_min_lex(s: str, k: int) -> str:
    """
    Removes k consecutive characters from the string `s` such that
    the resulting string is lexicographically smallest.
    """
    if k >= len(s):
        return ""
    
    min_str = s[:k] + s[k:]
    for i in range(len(s) - k + 1):
        candidate = s[:i] + s[i + k:]
        if candidate < min_str:
            min_str = candidate
    return min_str

result = remove_k_consecutive_min_lex(test_string, 3)
print(result)

adsddvrttttttdfdfvgbddddssssssaswsqwwwqefffefefggg
